In [23]:
from bs4 import BeautifulSoup
import unicodedata as un
import requests
import pandas as pd
import mechanize
import numpy as np
import matplotlib.pyplot as plt
import collections as cl

%matplotlib inline
#list all parameters by "print(plt.rcParams)" command
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['font.size'] = 15
plt.rcParams['axes.grid'] = True
plt.rcParams['figure.facecolor'] = 'w'

In [24]:
br = mechanize.Browser()
br.open('http://www.calrecycle.ca.gov/LGCentral/Reports/Jurisdiction/DiversionDisposal.aspx')
response = br.response()

# print response.geturl() # URL of the page we opened
# print response.info()   # headers
# print response.read()

In [25]:
br.select_form('aspnetForm')

In [26]:
# Set options for crawling
br.form['ctl00$cphBody$ddlYear'] = ['2007'] #['2012']
br.form['ctl00$cphBody$Geog'] = ['rdoGeoGeo']
br.form['ctl00$cphBody$newReport'] = ['rdoDisposal']
br.form['ctl00$cphBody$ddlReviewStatus'] = ['2']

In [27]:
# print(br.form['ctl00$cphBody$ddlYear'], br.form['ctl00$cphBody$Geog'], br.form['ctl00$cphBody$newReport'])

In [28]:
br.submit()
soup = BeautifulSoup(br.response().read())
# print br
# print soup

In [29]:
# Head to the section of interest
summary = soup.find("div",{'id':'ctl00_cphBody_pnlResultsNew'})
#summary = soup.find("table",{'id':'ctl00_cphBody_grdResultsOld'})
# Search for tables in the HTML
tables = summary.find_all('table')[0]
#tables = summary.find_all('tbody')
# tables[0]

In [30]:
# Create an empty data list
data = []

# Set rows as first indexed object in tables with a row
rows = tables.findAll('tr')

# Grab every HTML cell in every row
for tr in rows:
    cols = tr.findAll('td')
    # Check if text is in the row
    for td in cols:
        text = td.find(text=True) 
#         print text,
        data.append(text)

In [31]:
# data

In [32]:
Ser = pd.Series(data)
# Ser

In [33]:
# w/ option for review staus
for i in range(3710,3717):
    del Ser[i]

In [34]:
# Creating a dataframe
df = pd.DataFrame(Ser.reshape(371,10), columns = ['Jurisdiction','# of Programs implemented','AnnualReportReviewStatus','ReviewYears',
                          'JurisdictionReviewStatus','ComplianceStatus','PopulationTarget','DisposalPPDAnnual',
                          'EmploymentTarget','DisposalPPDAnnual'])
df

,Jurisdiction,# of Programs implemented,AnnualReportReviewStatus,ReviewYears,JurisdictionReviewStatus,ComplianceStatus,PopulationTarget,DisposalPPDAnnual,EmploymentTarget,DisposalPPDAnnual
0,Adelanto,35,Staff Reviewed,"2007, 2008, 2009",Approved,None,6.9,4.8,40.6,31.7
1,Agoura Hills,30,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,7.0,6.6,15.2,13.1
2,Alameda,45,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,5.5,3.8,21.4,13.8
3,Alameda-Unincorporated,44,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,4.9,3.9,19.8,16.0
4,Albany,41,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,5.0,2.9,19.3,11.0
5,Alhambra,34,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,3.8,3.3,14.4,11.9
6,Aliso Viejo,31,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,3.3,2.5,9.4,6.7
7,Alpine-Unincorporated,25,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,14.5,13.3,24.0,22.0
8,Amador County Integrated Solid Waste Managemen...,45,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,7.7,5.9,24.1,17.4
9,American Canyon,38,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,6.9,5.1,42.5,28.1


In [35]:
# w/o option for review staus
# del Ser[4190]
# del Ser[4191]
# del Ser[4192]
# del Ser[4193]
# del Ser[4194]
# del Ser[4195]
# del Ser[4196]

In [36]:
# # Creating a dataframe
# df = pd.DataFrame(Ser.reshape(419,10), columns = ['Jurisdiction','# of Programs implemented','AnnualReportReviewStatus','ReviewYears',
#                           'JurisdictionReviewStatus','ComplianceStatus','PopulationTarget','DisposalPPDAnnual',
#                           'EmploymentTarget','DisposalPPDAnnual'])
# df

In [37]:
def get_target_text(target_id):
    return soup.find(id=target_id).get_text().encode('utf-8').replace(',','')

In [38]:
disposal_amt_list = []
total_pop_list = []
total_employ_list = []
county_list = []

#Iterate the links
for link in br.links():
    if ".." and 'Detail' in link.url: #'Programs' for Detail Programs for recycling
#         print link.text, link.url
        request = br.click_link(link)
        response = br.follow_link(link)
        
        soup = BeautifulSoup(br.response().read())
#         table = soup.find_all('table')[0]
#         print table

        county_name = get_target_text('ctl00_cphBody_lblCountyName')
        total_disposal_amt = get_target_text('ctl00_cphBody_lblCalcAdjustedDisposalTon')
        total_pop = get_target_text('ctl00_cphBody_lblPopulation')
        total_employ = get_target_text('ctl00_cphBody_lblEmployment')
#         print county_name, total_disposal_amt, total_pop, total_employ
        
        disposal_amt_list.append(float(total_disposal_amt))
        total_pop_list.append(int(total_pop))
        total_employ_list.append(int(total_employ))
        county_list.append(county_name)


In [39]:
df.insert(0, "County", county_list)
df.insert(11, "TotalDisposalAcounty_listmount", disposal_amt_list)
df.insert(12, "TotalPopulation", total_pop_list)
df.insert(13, "TotalEmployment", total_employ_list)

In [40]:
df = df.drop_duplicates(['Jurisdiction'], take_last=True)
df

,County,Jurisdiction,# of Programs implemented,AnnualReportReviewStatus,ReviewYears,JurisdictionReviewStatus,ComplianceStatus,PopulationTarget,DisposalPPDAnnual,EmploymentTarget,DisposalPPDAnnual,TotalDisposalAcounty_listmount,TotalPopulation,TotalEmployment
0,San Bernardino,Adelanto,35,Staff Reviewed,"2007, 2008, 2009",Approved,None,6.9,4.8,40.6,31.7,23676.00,27088,4091
1,Los Angeles,Agoura Hills,30,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,7.0,6.6,15.2,13.1,27912.00,23208,11688
2,Alameda,Alameda,45,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,5.5,3.8,21.4,13.8,52612.77,75077,20878
3,Alameda,Alameda-Unincorporated,44,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,4.9,3.9,19.8,16.0,100313.74,139554,34366
4,Alameda,Albany,41,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,5.0,2.9,19.3,11.0,8949.87,16722,4470
5,Los Angeles,Alhambra,34,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,3.8,3.3,14.4,11.9,53778.88,88993,24748
6,Orange,Aliso Viejo,31,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,3.3,2.5,9.4,6.7,20386.04,44832,16721
7,Alpine,Alpine-Unincorporated,25,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,14.5,13.3,24.0,22.0,3022.00,1242,754
8,Amador,Amador County Integrated Solid Waste Managemen...,45,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,7.7,5.9,24.1,17.4,41214.00,38002,12948
9,Napa,American Canyon,38,Staff Reviewed,"2007, 2008, 2009, 2010, 2011",Approved,None,6.9,5.1,42.5,28.1,14909.00,15925,2908


In [41]:
consumed_2007_df = pd.read_csv('C:/Users/SangSeo/Desktop/NewmetData/data/2007/ECN_2007_US_42A1_with_ann.csv')
consumed_2007_df

,GEO.id,GEO.id2,GEO.display-label,NAICS.id,NAICS.display-label,OPTAX.id,OPTAX.display-label,YEAR.id,ESTAB,RCPTOT,PAYANN,PAYQTR1,EMP,OPEX,INVTOTB,INVTOTE,ADMIN_PCT,IMPUT_PCT
0,E600000US0600100562,600100562,"Alameda city, California",42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2007,62,2014726,146854,36390,1405,474756,469827,363439,4.1,0.5
1,E600000US0600100674,600100674,"Albany city, California",42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2007,14,10738,1688,405,44,2948,1046,1144,15.6,8.5
2,E600000US0600102980,600102980,"Ashland CDP, California",42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2007,7,D,D,D,b,D,D,D,D,D
3,E600000US0600106000,600106000,"Berkeley city, California",42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2007,118,620305,66804,17415,1394,146223,47809,50513,3.8,28.0
4,E600000US0600111964,600111964,"Castro Valley CDP, California",42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2007,22,30693,2417,589,54,4468,1266,1313,17.8,10.6
5,E600000US0600112902,600112902,"Cherryland CDP, California",42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2007,3,D,D,D,b,D,D,D,D,D
6,E600000US0600120018,600120018,"Dublin city, California",42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2007,57,251767,25501,6293,503,48629,21760,21774,33.7,9.9
7,E600000US0600122594,600122594,"Emeryville city, California",42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2007,65,599967,75929,18024,1163,139698,68222,69975,6.1,15.7
8,E600000US0600123350,600123350,"Fairview CDP, California",42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2007,2,D,D,D,a,D,D,D,D,D
9,E600000US0600126000,600126000,"Fremont city, California",42,Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2007,500,11312162,801894,193233,10709,1420208,844803,828661,10.7,2.9


In [42]:
consumed_2007_df.pop('GEO.id')
consumed_2007_df.pop('GEO.id2')
consumed_2007_df.pop('NAICS.id')
consumed_2007_df.pop('YEAR.id')

0      2007
1      2007
2      2007
3      2007
4      2007
5      2007
6      2007
7      2007
8      2007
9      2007
10     2007
11     2007
12     2007
13     2007
14     2007
15     2007
16     2007
17     2007
18     2007
19     2007
20     2007
21     2007
22     2007
23     2007
24     2007
25     2007
26     2007
27     2007
28     2007
29     2007
       ... 
601    2007
602    2007
603    2007
604    2007
605    2007
606    2007
607    2007
608    2007
609    2007
610    2007
611    2007
612    2007
613    2007
614    2007
615    2007
616    2007
617    2007
618    2007
619    2007
620    2007
621    2007
622    2007
623    2007
624    2007
625    2007
626    2007
627    2007
628    2007
629    2007
630    2007
Name: YEAR.id, dtype: int64

In [43]:
consumed_2007_df
# filtered = filtered.reset_index(drop=True)
# filtered

,GEO.display-label,NAICS.display-label,OPTAX.id,OPTAX.display-label,ESTAB,RCPTOT,PAYANN,PAYQTR1,EMP,OPEX,INVTOTB,INVTOTE,ADMIN_PCT,IMPUT_PCT
0,"Alameda city, California",Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",62,2014726,146854,36390,1405,474756,469827,363439,4.1,0.5
1,"Albany city, California",Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",14,10738,1688,405,44,2948,1046,1144,15.6,8.5
2,"Ashland CDP, California",Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",7,D,D,D,b,D,D,D,D,D
3,"Berkeley city, California",Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",118,620305,66804,17415,1394,146223,47809,50513,3.8,28.0
4,"Castro Valley CDP, California",Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",22,30693,2417,589,54,4468,1266,1313,17.8,10.6
5,"Cherryland CDP, California",Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",3,D,D,D,b,D,D,D,D,D
6,"Dublin city, California",Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",57,251767,25501,6293,503,48629,21760,21774,33.7,9.9
7,"Emeryville city, California",Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",65,599967,75929,18024,1163,139698,68222,69975,6.1,15.7
8,"Fairview CDP, California",Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",2,D,D,D,a,D,D,D,D,D
9,"Fremont city, California",Wholesale trade,10,"Merchant wholesalers, except manufacturers' sa...",500,11312162,801894,193233,10709,1420208,844803,828661,10.7,2.9


In [44]:
new_df = pd.read_csv('C:/Users/SangSeo/Desktop/NewmetData/data/PPDdisposal2007_2013_demogRaceHH.csv')
new_df

,Unnamed: 0,County,Jurisdiction,# of Programs implemented2007,ComplianceStatus2007,PopulationTarget2007,DisposalPPDAnnual2007,EmploymentTarget2007,DisposalPPDAnnualEmpl2007,# of Programs implemented2008,...,SomeOtherRace,TotalPopulationInHouseholds,Householder,Spouse,Children,ChildrenOwnChildUnder18y,OtherRelatives,OtherRelativesUnder18y,NonRelatives,NonRelativesUnmarriedPartner
0,0,San Bernardino,Adelanto,35,NaN,6.9,4.8,40.6,31.7,35,...,"8,337","30,012","7,809","4,000","12,641","9,693","3,806","1,816","1,756",827
1,1,Los Angeles,Agoura Hills,30,NaN,7.0,6.6,15.2,13.1,32,...,590,"20,242","7,327","4,565","6,721","4,682",745,155,884,299
2,2,Alameda,Alameda,45,NaN,5.5,3.8,21.4,13.8,45,...,"2,463","72,316","30,123","13,440","19,626","14,034","4,531","1,091","4,596","2,140"
3,3,Alameda,Alameda-Unincorporated,44,NaN,4.9,3.9,19.8,16.0,44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Alameda,Albany,41,NaN,5.0,2.9,19.3,11.0,41,...,607,"18,454","7,401","3,801","5,388","4,442",770,158,"1,094",464
5,5,Los Angeles,Alhambra,34,NaN,3.8,3.3,14.4,11.9,34,...,"10,805","82,475","29,217","13,679","22,761","13,224","11,006","2,231","5,812","1,553"
6,6,Orange,Aliso Viejo,31,NaN,3.3,2.5,9.4,6.7,34,...,"2,446","47,354","18,204","9,358","14,582","11,780","2,265",480,"2,945","1,193"
7,7,Alpine,Alpine-Unincorporated,25,NaN,14.5,13.3,24.0,22.0,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,Modoc,Alturas,33,NaN,6.8,7.3,14.5,15.3,33,...,118,"2,814","1,238",507,775,636,111,51,183,111
9,9,Amador,Amador County Integrated Solid Waste Managemen...,45,NaN,7.7,5.9,24.1,17.4,45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
new_df.pop('Unnamed: 0')
new_df.pop('ComplianceStatus2007')
new_df.pop('ComplianceStatus2008')
new_df.pop('ComplianceStatus2009')
new_df.pop('ComplianceStatus2010')
new_df.pop('ComplianceStatus2011')
new_df.pop('ComplianceStatus2012')
new_df.pop('ComplianceStatus2013')

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     NaN
25     NaN
26     NaN
27     NaN
28     NaN
29     NaN
      ... 
395    NaN
396    NaN
397    NaN
398    NaN
399    NaN
400    NaN
401    NaN
402    NaN
403    NaN
404    NaN
405    NaN
406    NaN
407    NaN
408    NaN
409    NaN
410    NaN
411    NaN
412    NaN
413    NaN
414    NaN
415    NaN
416    NaN
417    NaN
418    NaN
419    NaN
420    NaN
421    NaN
422    NaN
423    NaN
424    NaN
Name: ComplianceStatus2013, dtype: object

In [46]:
new_df#['# of Programs implemented2007']

,County,Jurisdiction,# of Programs implemented2007,PopulationTarget2007,DisposalPPDAnnual2007,EmploymentTarget2007,DisposalPPDAnnualEmpl2007,# of Programs implemented2008,PopulationTarget2008,DisposalPPDAnnual2008,...,SomeOtherRace,TotalPopulationInHouseholds,Householder,Spouse,Children,ChildrenOwnChildUnder18y,OtherRelatives,OtherRelativesUnder18y,NonRelatives,NonRelativesUnmarriedPartner
0,San Bernardino,Adelanto,35,6.9,4.8,40.6,31.7,35,6.9,3.8,...,"8,337","30,012","7,809","4,000","12,641","9,693","3,806","1,816","1,756",827
1,Los Angeles,Agoura Hills,30,7.0,6.6,15.2,13.1,32,7.0,5.7,...,590,"20,242","7,327","4,565","6,721","4,682",745,155,884,299
2,Alameda,Alameda,45,5.5,3.8,21.4,13.8,45,5.5,3.6,...,"2,463","72,316","30,123","13,440","19,626","14,034","4,531","1,091","4,596","2,140"
3,Alameda,Alameda-Unincorporated,44,4.9,3.9,19.8,16.0,44,4.9,3.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alameda,Albany,41,5.0,2.9,19.3,11.0,41,5.0,2.3,...,607,"18,454","7,401","3,801","5,388","4,442",770,158,"1,094",464
5,Los Angeles,Alhambra,34,3.8,3.3,14.4,11.9,34,3.8,3.2,...,"10,805","82,475","29,217","13,679","22,761","13,224","11,006","2,231","5,812","1,553"
6,Orange,Aliso Viejo,31,3.3,2.5,9.4,6.7,34,3.3,2.2,...,"2,446","47,354","18,204","9,358","14,582","11,780","2,265",480,"2,945","1,193"
7,Alpine,Alpine-Unincorporated,25,14.5,13.3,24.0,22.0,25,14.5,11.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Modoc,Alturas,33,6.8,7.3,14.5,15.3,33,6.8,6.5,...,118,"2,814","1,238",507,775,636,111,51,183,111
9,Amador,Amador County Integrated Solid Waste Managemen...,45,7.7,5.9,24.1,17.4,45,7.7,5.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
new_df.columns

Index([u'County', u'Jurisdiction', u'# of Programs implemented2007',
       u'PopulationTarget2007', u'DisposalPPDAnnual2007',
       u'EmploymentTarget2007', u'DisposalPPDAnnualEmpl2007',
       u'# of Programs implemented2008', u'PopulationTarget2008',
       u'DisposalPPDAnnual2008', u'EmploymentTarget2008',
       u'DisposalPPDAnnualEmpl2008', u'# of Programs implemented2009',
       u'PopulationTarget2009', u'DisposalPPDAnnual2009',
       u'EmploymentTarget2009', u'DisposalPPDAnnualEmpl2009',
       u'# of Programs implemented2010', u'PopulationTarget2010',
       u'DisposalPPDAnnual2010', u'EmploymentTarget2010',
       u'DisposalPPDAnnualEmpl2010', u'# of Programs implemented2011',
       u'PopulationTarget2011', u'DisposalPPDAnnual2011',
       u'EmploymentTarget2011', u'DisposalPPDAnnualEmpl2011',
       u'# of Programs implemented2012', u'PopulationTarget2012',
       u'DisposalPPDAnnual2012', u'EmploymentTarget2012',
       u'DisposalPPDAnnualEmpl2012', u'# of Programs impl

In [48]:
new_df=new_df.rename(columns = {'# of Programs implemented2007':'Num_RecyclingProg_2007', 
                               '# of Programs implemented2008':'Num_RecyclingProg_2008',
                               '# of Programs implemented2009':'Num_RecyclingProg_2009',
                               '# of Programs implemented2010':'Num_RecyclingProg_2010',
                               '# of Programs implemented2011':'Num_RecyclingProg_2011',
                               '# of Programs implemented2012':'Num_RecyclingProg_2012',
                               '# of Programs implemented2013':'Num_RecyclingProg_2013'})
new_df.columns

Index([u'County', u'Jurisdiction', u'Num_RecyclingProg_2007',
       u'PopulationTarget2007', u'DisposalPPDAnnual2007',
       u'EmploymentTarget2007', u'DisposalPPDAnnualEmpl2007',
       u'Num_RecyclingProg_2008', u'PopulationTarget2008',
       u'DisposalPPDAnnual2008', u'EmploymentTarget2008',
       u'DisposalPPDAnnualEmpl2008', u'Num_RecyclingProg_2009',
       u'PopulationTarget2009', u'DisposalPPDAnnual2009',
       u'EmploymentTarget2009', u'DisposalPPDAnnualEmpl2009',
       u'Num_RecyclingProg_2010', u'PopulationTarget2010',
       u'DisposalPPDAnnual2010', u'EmploymentTarget2010',
       u'DisposalPPDAnnualEmpl2010', u'Num_RecyclingProg_2011',
       u'PopulationTarget2011', u'DisposalPPDAnnual2011',
       u'EmploymentTarget2011', u'DisposalPPDAnnualEmpl2011',
       u'Num_RecyclingProg_2012', u'PopulationTarget2012',
       u'DisposalPPDAnnual2012', u'EmploymentTarget2012',
       u'DisposalPPDAnnualEmpl2012', u'Num_RecyclingProg_2013',
       u'PopulationTarget2013', u'D

In [49]:
new_df=new_df.rename(columns = {'PopulationTarget2007':'Target_ResPPD_2007', 
                               'PopulationTarget2008':'Target_ResPPD_2008',
                               'PopulationTarget2009':'Target_ResPPD_2009',
                               'PopulationTarget2010':'Target_ResPPD_2010',
                               'PopulationTarget2011':'Target_ResPPD_2011',
                               'PopulationTarget2012':'Target_ResPPD_2012',
                               'PopulationTarget2013':'Target_ResPPD_2013'})
new_df.columns

Index([u'County', u'Jurisdiction', u'Num_RecyclingProg_2007',
       u'Target_ResPPD_2007', u'DisposalPPDAnnual2007',
       u'EmploymentTarget2007', u'DisposalPPDAnnualEmpl2007',
       u'Num_RecyclingProg_2008', u'Target_ResPPD_2008',
       u'DisposalPPDAnnual2008', u'EmploymentTarget2008',
       u'DisposalPPDAnnualEmpl2008', u'Num_RecyclingProg_2009',
       u'Target_ResPPD_2009', u'DisposalPPDAnnual2009',
       u'EmploymentTarget2009', u'DisposalPPDAnnualEmpl2009',
       u'Num_RecyclingProg_2010', u'Target_ResPPD_2010',
       u'DisposalPPDAnnual2010', u'EmploymentTarget2010',
       u'DisposalPPDAnnualEmpl2010', u'Num_RecyclingProg_2011',
       u'Target_ResPPD_2011', u'DisposalPPDAnnual2011',
       u'EmploymentTarget2011', u'DisposalPPDAnnualEmpl2011',
       u'Num_RecyclingProg_2012', u'Target_ResPPD_2012',
       u'DisposalPPDAnnual2012', u'EmploymentTarget2012',
       u'DisposalPPDAnnualEmpl2012', u'Num_RecyclingProg_2013',
       u'Target_ResPPD_2013', u'DisposalPPDAnnu

In [50]:
new_df=new_df.rename(columns = {'DisposalPPDAnnual2007':'Annual_ResPPD_2007', 
                               'DisposalPPDAnnual2008':'Annual_ResPPD_2008',
                               'DisposalPPDAnnual2009':'Annual_ResPPD_2009',
                               'DisposalPPDAnnual2010':'Annual_ResPPD_2010',
                               'DisposalPPDAnnual2011':'Annual_ResPPD_2011',
                               'DisposalPPDAnnual2012':'Annual_ResPPD_2012',
                               'DisposalPPDAnnual2013':'Annual_ResPPD_2013'})
new_df.columns

Index([u'County', u'Jurisdiction', u'Num_RecyclingProg_2007',
       u'Target_ResPPD_2007', u'Annual_ResPPD_2007', u'EmploymentTarget2007',
       u'DisposalPPDAnnualEmpl2007', u'Num_RecyclingProg_2008',
       u'Target_ResPPD_2008', u'Annual_ResPPD_2008', u'EmploymentTarget2008',
       u'DisposalPPDAnnualEmpl2008', u'Num_RecyclingProg_2009',
       u'Target_ResPPD_2009', u'Annual_ResPPD_2009', u'EmploymentTarget2009',
       u'DisposalPPDAnnualEmpl2009', u'Num_RecyclingProg_2010',
       u'Target_ResPPD_2010', u'Annual_ResPPD_2010', u'EmploymentTarget2010',
       u'DisposalPPDAnnualEmpl2010', u'Num_RecyclingProg_2011',
       u'Target_ResPPD_2011', u'Annual_ResPPD_2011', u'EmploymentTarget2011',
       u'DisposalPPDAnnualEmpl2011', u'Num_RecyclingProg_2012',
       u'Target_ResPPD_2012', u'Annual_ResPPD_2012', u'EmploymentTarget2012',
       u'DisposalPPDAnnualEmpl2012', u'Num_RecyclingProg_2013',
       u'Target_ResPPD_2013', u'Annual_ResPPD_2013', u'EmploymentTarget2013',
       u

In [51]:
new_df=new_df.rename(columns = {'EmploymentTarget2007':'Target_EmpPPD_2007', 
                               'EmploymentTarget2008':'Target_EmpPPD_2008',
                               'EmploymentTarget2009':'Target_EmpPPD_2009',
                               'EmploymentTarget2010':'Target_EmpPPD_2010',
                               'EmploymentTarget2011':'Target_EmpPPD_2011',
                               'EmploymentTarget2012':'Target_EmpPPD_2012',
                               'EmploymentTarget2013':'Target_EmpPPD_2013'})
new_df.columns

Index([u'County', u'Jurisdiction', u'Num_RecyclingProg_2007',
       u'Target_ResPPD_2007', u'Annual_ResPPD_2007', u'Target_EmpPPD_2007',
       u'DisposalPPDAnnualEmpl2007', u'Num_RecyclingProg_2008',
       u'Target_ResPPD_2008', u'Annual_ResPPD_2008', u'Target_EmpPPD_2008',
       u'DisposalPPDAnnualEmpl2008', u'Num_RecyclingProg_2009',
       u'Target_ResPPD_2009', u'Annual_ResPPD_2009', u'Target_EmpPPD_2009',
       u'DisposalPPDAnnualEmpl2009', u'Num_RecyclingProg_2010',
       u'Target_ResPPD_2010', u'Annual_ResPPD_2010', u'Target_EmpPPD_2010',
       u'DisposalPPDAnnualEmpl2010', u'Num_RecyclingProg_2011',
       u'Target_ResPPD_2011', u'Annual_ResPPD_2011', u'Target_EmpPPD_2011',
       u'DisposalPPDAnnualEmpl2011', u'Num_RecyclingProg_2012',
       u'Target_ResPPD_2012', u'Annual_ResPPD_2012', u'Target_EmpPPD_2012',
       u'DisposalPPDAnnualEmpl2012', u'Num_RecyclingProg_2013',
       u'Target_ResPPD_2013', u'Annual_ResPPD_2013', u'Target_EmpPPD_2013',
       u'DisposalPPDAn

In [52]:
new_df=new_df.rename(columns = {'DisposalPPDAnnualEmpl2007':'Annual_EmpPPD_2007', 
                               'DisposalPPDAnnualEmpl2008':'Annual_EmpPPD_2008',
                               'DisposalPPDAnnualEmpl2009':'Annual_EmpPPD_2009',
                               'DisposalPPDAnnualEmpl2010':'Annual_EmpPPD_2010',
                               'DisposalPPDAnnualEmpl2011':'Annual_EmpPPD_2011',
                               'DisposalPPDAnnualEmpl2012':'Annual_EmpPPD_2012',
                               'DisposalPPDAnnualEmpl2013':'Annual_EmpPPD_2013'})
new_df.columns

Index([u'County', u'Jurisdiction', u'Num_RecyclingProg_2007',
       u'Target_ResPPD_2007', u'Annual_ResPPD_2007', u'Target_EmpPPD_2007',
       u'Annual_EmpPPD_2007', u'Num_RecyclingProg_2008', u'Target_ResPPD_2008',
       u'Annual_ResPPD_2008', u'Target_EmpPPD_2008', u'Annual_EmpPPD_2008',
       u'Num_RecyclingProg_2009', u'Target_ResPPD_2009', u'Annual_ResPPD_2009',
       u'Target_EmpPPD_2009', u'Annual_EmpPPD_2009', u'Num_RecyclingProg_2010',
       u'Target_ResPPD_2010', u'Annual_ResPPD_2010', u'Target_EmpPPD_2010',
       u'Annual_EmpPPD_2010', u'Num_RecyclingProg_2011', u'Target_ResPPD_2011',
       u'Annual_ResPPD_2011', u'Target_EmpPPD_2011', u'Annual_EmpPPD_2011',
       u'Num_RecyclingProg_2012', u'Target_ResPPD_2012', u'Annual_ResPPD_2012',
       u'Target_EmpPPD_2012', u'Annual_EmpPPD_2012', u'Num_RecyclingProg_2013',
       u'Target_ResPPD_2013', u'Annual_ResPPD_2013', u'Target_EmpPPD_2013',
       u'Annual_EmpPPD_2013', u'Population', u'Male', u'Female',
       u'Medi

In [53]:
# new_df.dtypes

In [54]:
new_df['Num_RecyclingProg_2007'] = new_df['Num_RecyclingProg_2007'].fillna(0).astype(int)
new_df['Num_RecyclingProg_2008'] = new_df['Num_RecyclingProg_2008'].fillna(0).astype(int)
new_df['Num_RecyclingProg_2009'] = new_df['Num_RecyclingProg_2009'].fillna(0).astype(int)
new_df['Num_RecyclingProg_2010'] = new_df['Num_RecyclingProg_2010'].fillna(0).astype(int)
new_df['Num_RecyclingProg_2011'] = new_df['Num_RecyclingProg_2011'].fillna(0).astype(int)
new_df['Num_RecyclingProg_2012'] = new_df['Num_RecyclingProg_2012'].fillna(0).astype(int)
new_df['Num_RecyclingProg_2013'] = new_df['Num_RecyclingProg_2013'].fillna(0).astype(int)
# df[['two', 'three']] = df[['two', 'three']].astype(float)

In [55]:
new_df.dtypes

County                            object
Jurisdiction                      object
Num_RecyclingProg_2007             int32
Target_ResPPD_2007               float64
Annual_ResPPD_2007               float64
Target_EmpPPD_2007               float64
Annual_EmpPPD_2007               float64
Num_RecyclingProg_2008             int32
Target_ResPPD_2008               float64
Annual_ResPPD_2008               float64
Target_EmpPPD_2008               float64
Annual_EmpPPD_2008               float64
Num_RecyclingProg_2009             int32
Target_ResPPD_2009               float64
Annual_ResPPD_2009               float64
Target_EmpPPD_2009               float64
Annual_EmpPPD_2009               float64
Num_RecyclingProg_2010             int32
Target_ResPPD_2010               float64
Annual_ResPPD_2010               float64
Target_EmpPPD_2010               float64
Annual_EmpPPD_2010               float64
Num_RecyclingProg_2011             int32
Target_ResPPD_2011               float64
Annual_ResPPD_20

In [56]:
# new_df.dtypes

In [57]:
# remove ',' from column values
for col in new_df.columns:
    if new_df[col].dtype == np.object:
        new_df[col] = new_df[col].str.replace(',','')

In [58]:
# remove '%' from column values
new_df['PercentPopulationFemale'] = new_df['PercentPopulationFemale'].str.replace('%','')
new_df['PercentPopulationLessThan18'] = new_df['PercentPopulationLessThan18'].str.replace('%','')

In [59]:
new_df.sort(columns=['Jurisdiction']) #'County', 

,County,Jurisdiction,Num_RecyclingProg_2007,Target_ResPPD_2007,Annual_ResPPD_2007,Target_EmpPPD_2007,Annual_EmpPPD_2007,Num_RecyclingProg_2008,Target_ResPPD_2008,Annual_ResPPD_2008,...,SomeOtherRace,TotalPopulationInHouseholds,Householder,Spouse,Children,ChildrenOwnChildUnder18y,OtherRelatives,OtherRelativesUnder18y,NonRelatives,NonRelativesUnmarriedPartner
0,San Bernardino,Adelanto,35,6.9,4.8,40.6,31.7,35,6.9,3.8,...,8337,30012,7809,4000,12641,9693,3806,1816,1756,827
1,Los Angeles,Agoura Hills,30,7.0,6.6,15.2,13.1,32,7.0,5.7,...,590,20242,7327,4565,6721,4682,745,155,884,299
2,Alameda,Alameda,45,5.5,3.8,21.4,13.8,45,5.5,3.6,...,2463,72316,30123,13440,19626,14034,4531,1091,4596,2140
3,Alameda,Alameda-Unincorporated,44,4.9,3.9,19.8,16.0,44,4.9,3.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alameda,Albany,41,5.0,2.9,19.3,11.0,41,5.0,2.3,...,607,18454,7401,3801,5388,4442,770,158,1094,464
5,Los Angeles,Alhambra,34,3.8,3.3,14.4,11.9,34,3.8,3.2,...,10805,82475,29217,13679,22761,13224,11006,2231,5812,1553
6,Orange,Aliso Viejo,31,3.3,2.5,9.4,6.7,34,3.3,2.2,...,2446,47354,18204,9358,14582,11780,2265,480,2945,1193
7,Alpine,Alpine-Unincorporated,25,14.5,13.3,24.0,22.0,25,14.5,11.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Modoc,Alturas,33,6.8,7.3,14.5,15.3,33,6.8,6.5,...,118,2814,1238,507,775,636,111,51,183,111
9,Amador,Amador County Integrated Solid Waste Managemen...,45,7.7,5.9,24.1,17.4,45,7.7,5.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
mike_df = pd.read_csv('C:/Users/SangSeo/Downloads/vals_with_counties.csv')
mike_df.pop('Unnamed: 0')

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23
24    24
25    25
26    26
27    27
28    28
29    29
30    30
31    31
32    32
33    33
34    34
35    35
36    36
37    37
38    38
39    39
40    40
41    41
42    42
43    43
44    44
45    45
46    46
47    47
48    48
49    49
50    50
51    51
52    52
53    53
54    54
55    55
56    56
57    57
Name: Unnamed: 0, dtype: int64

In [61]:
print new_df.dtypes
mike_df.dtypes

County                            object
Jurisdiction                      object
Num_RecyclingProg_2007             int32
Target_ResPPD_2007               float64
Annual_ResPPD_2007               float64
Target_EmpPPD_2007               float64
Annual_EmpPPD_2007               float64
Num_RecyclingProg_2008             int32
Target_ResPPD_2008               float64
Annual_ResPPD_2008               float64
Target_EmpPPD_2008               float64
Annual_EmpPPD_2008               float64
Num_RecyclingProg_2009             int32
Target_ResPPD_2009               float64
Annual_ResPPD_2009               float64
Target_EmpPPD_2009               float64
Annual_EmpPPD_2009               float64
Num_RecyclingProg_2010             int32
Target_ResPPD_2010               float64
Annual_ResPPD_2010               float64
Target_EmpPPD_2010               float64
Annual_EmpPPD_2010               float64
Num_RecyclingProg_2011             int32
Target_ResPPD_2011               float64
Annual_ResPPD_20

MedianAge                float64
MedianHouseholdIncome    float64
MeanHouseholdIncome      float64
PercentBachDegHghr       float64
County                    object
dtype: object

In [62]:
mike_df['County'] = mike_df['County'].str.strip()
mike_df['County']

0             Alameda
1              Alpine
2              Amador
3               Butte
4           Calaveras
5              Colusa
6        Contra Costa
7           Del Norte
8           El Dorado
9              Fresno
10              Glenn
11           Humboldt
12           Imperial
13               Inyo
14               Kern
15              Kings
16               Lake
17             Lassen
18        Los Angeles
19             Madera
20              Marin
21           Mariposa
22          Mendocino
23             Merced
24              Modoc
25               Mono
26           Monterey
27               Napa
28             Nevada
29             Orange
30             Placer
31             Plumas
32          Riverside
33         Sacramento
34         San Benito
35     San Bernardino
36          San Diego
37      San Francisco
38        San Joaquin
39    San Luis Obispo
40          San Mateo
41      Santa Barbara
42        Santa Clara
43         Santa Cruz
44             Shasta
45        

In [63]:
# join_df = new_df.join(mike_df, on='County', how="left", lsuffix='County', rsuffix='County')
join_df = pd.merge(new_df, mike_df, on='County', how='left')
# join_df.sort(columns=['County'])
join_df.columns
# print "Los Angeles" == "Los Angeles"
# for value in new_df['County'].values:
#     for value2 in mike_df['County'].values:
#         if value == value2:
#             print 'True'
#         print value, value2

Index([u'County', u'Jurisdiction', u'Num_RecyclingProg_2007',
       u'Target_ResPPD_2007', u'Annual_ResPPD_2007', u'Target_EmpPPD_2007',
       u'Annual_EmpPPD_2007', u'Num_RecyclingProg_2008', u'Target_ResPPD_2008',
       u'Annual_ResPPD_2008', u'Target_EmpPPD_2008', u'Annual_EmpPPD_2008',
       u'Num_RecyclingProg_2009', u'Target_ResPPD_2009', u'Annual_ResPPD_2009',
       u'Target_EmpPPD_2009', u'Annual_EmpPPD_2009', u'Num_RecyclingProg_2010',
       u'Target_ResPPD_2010', u'Annual_ResPPD_2010', u'Target_EmpPPD_2010',
       u'Annual_EmpPPD_2010', u'Num_RecyclingProg_2011', u'Target_ResPPD_2011',
       u'Annual_ResPPD_2011', u'Target_EmpPPD_2011', u'Annual_EmpPPD_2011',
       u'Num_RecyclingProg_2012', u'Target_ResPPD_2012', u'Annual_ResPPD_2012',
       u'Target_EmpPPD_2012', u'Annual_EmpPPD_2012', u'Num_RecyclingProg_2013',
       u'Target_ResPPD_2013', u'Annual_ResPPD_2013', u'Target_EmpPPD_2013',
       u'Annual_EmpPPD_2013', u'Population', u'Male', u'Female',
       u'Medi

In [64]:
join_df.to_csv('ppddisposal2007_2013_demogracehh_final.csv')

In [65]:
county_1 = new_df['County'].values[3]
county_2 = mike_df['County'].values[0]

print type(county_1), type(county_2)
print county_1, county_2
print county_1 == county_2
print county_1 == county_2.strip()
# print new_df['County'].values[3].strip() is mike_df['County'].values[0].strip()
# pd.merge(new_df, mike_df, how='outer', on='County')

<type 'str'> <type 'str'>
Alameda Alameda
True
True


In [66]:
# write csv file
new_df.to_csv('./ppddisposal2007_2013_demogracehh_steve.csv')

In [67]:
excel_df = pd.ExcelFile(io='merged_data.xlsx')
# excel_df.sheet_names
df = excel_df.parse("Sheet1")
df

,Geography,Population,Male,Female,MedianAgeYears,MaleMedianAgeYears,FemaleMedianAgeYears,AverageHouseholdSize,AverageFamilySize,PersonsUnder5Years,...,SomeOtherRace,TotalPopulationInHouseholds,Householder,Spouse,Children,ChildrenOwnChildUnder18y,OtherRelatives,OtherRelativesUnder18y,NonRelatives,NonRelativesUnmarriedPartner
0,California,37253956,18517830,18736126,35.2,34.0,36.3,2.896772,3.45,2531333,...,6317372,36434140,12577498,6213310,11308643,7839242,3611597,1221251,2723092,902967
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alameda County,1510271,740573,769698,36.6,35.6,37.5,2.701754,3.30,97652,...,162540,1472829,545138,253685,421917,296810,134770,36934,117319,39567
3,Alameda city,73812,35315,38497,40.7,39.5,41.9,2.400691,3.06,4182,...,2463,72316,30123,13440,19626,14034,4531,1091,4596,2140
4,Albany city,18539,8821,9718,37.0,35.8,38.2,2.493447,3.00,1409,...,607,18454,7401,3801,5388,4442,770,158,1094,464
5,Ashland CDP,21925,10691,11234,31.4,30.6,32.2,2.990234,3.52,1907,...,5124,21739,7270,2786,7095,5136,2722,802,1866,693
6,Berkeley city,112580,55031,57549,31.0,30.2,32.0,2.166699,2.81,4136,...,4994,99731,46029,13569,16722,12535,3792,991,19619,3892
7,Castro Valley CDP,61388,29827,31561,41.2,40.2,42.2,2.693037,3.15,3374,...,3757,60184,22348,12140,18465,12926,4034,957,3197,1341
8,Cherryland CDP,14728,7433,7295,32.3,31.9,32.7,3.073875,3.64,1225,...,4016,14272,4643,1850,4541,3313,1763,532,1475,497
9,Dublin city,46036,23983,22053,35.3,34.7,35.9,2.699792,3.19,3404,...,2458,40262,14913,8615,12350,9685,2305,495,2079,917


In [68]:
cl.Counter(pd.isnull(df['Geography']))
df = df[pd.notnull(df['Geography'])]
df

,Geography,Population,Male,Female,MedianAgeYears,MaleMedianAgeYears,FemaleMedianAgeYears,AverageHouseholdSize,AverageFamilySize,PersonsUnder5Years,...,SomeOtherRace,TotalPopulationInHouseholds,Householder,Spouse,Children,ChildrenOwnChildUnder18y,OtherRelatives,OtherRelativesUnder18y,NonRelatives,NonRelativesUnmarriedPartner
0,California,37253956,18517830,18736126,35.2,34.0,36.3,2.896772,3.45,2531333,...,6317372,36434140,12577498,6213310,11308643,7839242,3611597,1221251,2723092,902967
2,Alameda County,1510271,740573,769698,36.6,35.6,37.5,2.701754,3.30,97652,...,162540,1472829,545138,253685,421917,296810,134770,36934,117319,39567
3,Alameda city,73812,35315,38497,40.7,39.5,41.9,2.400691,3.06,4182,...,2463,72316,30123,13440,19626,14034,4531,1091,4596,2140
4,Albany city,18539,8821,9718,37.0,35.8,38.2,2.493447,3.00,1409,...,607,18454,7401,3801,5388,4442,770,158,1094,464
5,Ashland CDP,21925,10691,11234,31.4,30.6,32.2,2.990234,3.52,1907,...,5124,21739,7270,2786,7095,5136,2722,802,1866,693
6,Berkeley city,112580,55031,57549,31.0,30.2,32.0,2.166699,2.81,4136,...,4994,99731,46029,13569,16722,12535,3792,991,19619,3892
7,Castro Valley CDP,61388,29827,31561,41.2,40.2,42.2,2.693037,3.15,3374,...,3757,60184,22348,12140,18465,12926,4034,957,3197,1341
8,Cherryland CDP,14728,7433,7295,32.3,31.9,32.7,3.073875,3.64,1225,...,4016,14272,4643,1850,4541,3313,1763,532,1475,497
9,Dublin city,46036,23983,22053,35.3,34.7,35.9,2.699792,3.19,3404,...,2458,40262,14913,8615,12350,9685,2305,495,2079,917
10,Emeryville city,10080,4983,5097,35.0,34.8,35.2,1.757464,2.61,424,...,348,10007,5694,1240,1240,908,470,97,1363,600


In [69]:
new_df

,County,Jurisdiction,Num_RecyclingProg_2007,Target_ResPPD_2007,Annual_ResPPD_2007,Target_EmpPPD_2007,Annual_EmpPPD_2007,Num_RecyclingProg_2008,Target_ResPPD_2008,Annual_ResPPD_2008,...,SomeOtherRace,TotalPopulationInHouseholds,Householder,Spouse,Children,ChildrenOwnChildUnder18y,OtherRelatives,OtherRelativesUnder18y,NonRelatives,NonRelativesUnmarriedPartner
0,San Bernardino,Adelanto,35,6.9,4.8,40.6,31.7,35,6.9,3.8,...,8337,30012,7809,4000,12641,9693,3806,1816,1756,827
1,Los Angeles,Agoura Hills,30,7.0,6.6,15.2,13.1,32,7.0,5.7,...,590,20242,7327,4565,6721,4682,745,155,884,299
2,Alameda,Alameda,45,5.5,3.8,21.4,13.8,45,5.5,3.6,...,2463,72316,30123,13440,19626,14034,4531,1091,4596,2140
3,Alameda,Alameda-Unincorporated,44,4.9,3.9,19.8,16.0,44,4.9,3.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alameda,Albany,41,5.0,2.9,19.3,11.0,41,5.0,2.3,...,607,18454,7401,3801,5388,4442,770,158,1094,464
5,Los Angeles,Alhambra,34,3.8,3.3,14.4,11.9,34,3.8,3.2,...,10805,82475,29217,13679,22761,13224,11006,2231,5812,1553
6,Orange,Aliso Viejo,31,3.3,2.5,9.4,6.7,34,3.3,2.2,...,2446,47354,18204,9358,14582,11780,2265,480,2945,1193
7,Alpine,Alpine-Unincorporated,25,14.5,13.3,24.0,22.0,25,14.5,11.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Modoc,Alturas,33,6.8,7.3,14.5,15.3,33,6.8,6.5,...,118,2814,1238,507,775,636,111,51,183,111
9,Amador,Amador County Integrated Solid Waste Managemen...,45,7.7,5.9,24.1,17.4,45,7.7,5.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
missing_counties = new_df['Jurisdiction'][pd.isnull(new_df['Population'])]
counties = [name for name in missing_counties.values if not '-Unincorporated' in name and not 'Waste' in name and not 'Agency' in name]
print len(missing_counties), missing_counties.values
len(counties), counties


94 ['Alameda-Unincorporated' 'Alpine-Unincorporated'
 'Amador County Integrated Solid Waste Management Agency' 'Apple Valley'
 'Atherton' 'Butte County Regional Waste Management Authority'
 'Calaveras County Regional Agency' 'California City' 'Cathedral City'
 'Colma' 'Colusa County Regional Agency'
 'Consolidated Waste Management Authority'
 'Contra Costa/Ironhouse/Oakley Regional Agency' 'Culver City' 'Daly City'
 'Danville' 'Del Norte Solid Waste Management Authority'
 'El Dorado-Unincorporated' 'Foster City' 'Fresno-Unincorporated'
 'Glenn County Waste Management Regional Agency' 'Hillsborough'
 'Humboldt-Unincorporated' 'Imperial Valley Resource Management Agency'
 'Inyo Regional Waste Management Agency' 'Kern-Unincorporated' 'King City'
 'Kings Waste and Recycling Authority' 'La Canada Flintridge'
 'Lake-Unincorporated' 'Lassen Regional Solid Waste Management Authority'
 'Loomis' 'Los Altos Hills'
 'Los Angeles Area Integrated Waste Management Authority'
 'Los Angeles-Unincorpora

(32,
 ['Apple Valley',
  'Atherton',
  'California City',
  'Cathedral City',
  'Colma',
  'Culver City',
  'Daly City',
  'Danville',
  'Foster City',
  'Hillsborough',
  'King City',
  'La Canada Flintridge',
  'Loomis',
  'Los Altos Hills',
  'Los Gatos',
  'Mammoth Lakes',
  'Moraga',
  'National City',
  'Nevada City',
  'Portola Valley',
  'Redwood City',
  'San Buenaventura',
  'Sand City',
  'Suisun City',
  'Temple City',
  'Truckee',
  'Union City',
  'Woodside',
  'Yucca Valley',
  'Paradise',
  'Eastvale',
  'Jurupa Valley'])

In [71]:
nan_df = new_df[pd.isnull(new_df['Population'])]
nan_df

,County,Jurisdiction,Num_RecyclingProg_2007,Target_ResPPD_2007,Annual_ResPPD_2007,Target_EmpPPD_2007,Annual_EmpPPD_2007,Num_RecyclingProg_2008,Target_ResPPD_2008,Annual_ResPPD_2008,...,SomeOtherRace,TotalPopulationInHouseholds,Householder,Spouse,Children,ChildrenOwnChildUnder18y,OtherRelatives,OtherRelativesUnder18y,NonRelatives,NonRelativesUnmarriedPartner
3,Alameda,Alameda-Unincorporated,44,4.9,3.9,19.8,16.0,44,4.9,3.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Alpine,Alpine-Unincorporated,25,14.5,13.3,24.0,22.0,25,14.5,11.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Amador,Amador County Integrated Solid Waste Managemen...,45,7.7,5.9,24.1,17.4,45,7.7,5.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,San Bernardino,Apple Valley,36,6.6,5.1,36.0,27.3,36,6.6,4.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,San Mateo,Atherton,41,11.4,6.0,48.9,23.6,41,11.4,5.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,Butte,Butte County Regional Waste Management Authority,42,4.9,4.9,24.9,26.7,43,4.9,4.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,Calaveras,Calaveras County Regional Agency,42,4.0,5.9,20.1,29.9,42,4.0,4.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,Kern,California City,39,3.7,4.5,49.6,35.3,35,3.7,3.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,Riverside,Cathedral City,41,6.9,5.2,31.7,23.1,41,6.9,5.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,San Mateo,Colma,32,37.1,23.8,12.5,9.2,32,37.1,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df.columns

Index([                    u'Geography',                    u'Population',
                                u'Male',                        u'Female',
                      u'MedianAgeYears',            u'MaleMedianAgeYears',
                u'FemaleMedianAgeYears',          u'AverageHouseholdSize',
                   u'AverageFamilySize',            u'PersonsUnder5Years',
                 u'PersonsUnder18Years',              u'PersonsAge21Plus',
                    u'PersonsAge55Plus',              u'PersonsAge60Plus',
                    u'PersonsAge65Plus',       u'PercentPopulationFemale',
         u'PercentPopulationLessThan18',                   u'PopulationR',
            u'TotalPopulationOfOneRace',                         u'White',
              u'BlackOrAfricanAmerican', u'AmericanIndianAndAlaskaNative',
                          u'TotalAsian',                   u'AsianIndian',
                             u'Chinese',                      u'Filipino',
                         

In [73]:
nan_df['Jurisdiction']

3                                 Alameda-Unincorporated
7                                  Alpine-Unincorporated
9      Amador County Integrated Solid Waste Managemen...
13                                          Apple Valley
17                                              Atherton
44      Butte County Regional Waste Management Authority
46                      Calaveras County Regional Agency
47                                       California City
57                                        Cathedral City
71                                                 Colma
73                         Colusa County Regional Agency
77               Consolidated Waste Management Authority
78         Contra Costa/Ironhouse/Oakley Regional Agency
84                                           Culver City
87                                             Daly City
89                                              Danville
92            Del Norte Solid Waste Management Authority
102                            

In [74]:
df['Geography']

0                                   California 
2                                Alameda County
3                                 Alameda city 
4                                  Albany city 
5                                  Ashland CDP 
6                                Berkeley city 
7                            Castro Valley CDP 
8                               Cherryland CDP 
9                                  Dublin city 
10                             Emeryville city 
11                                Fairview CDP 
12                                Fremont city 
13                                Hayward city 
14                              Livermore city 
15                                 Newark city 
16                                Oakland city 
17                               Piedmont city 
18                             Pleasanton city 
19                            San Leandro city 
20                             San Lorenzo CDP 
21                                   Sun